# Builder123D Sandbox

Important Links:
* [Builder123D Github](https://github.com/gumyr/build123d)
    * [Documentation](https://build123d.readthedocs.io/en/latest)
* [VSCode Visualization Plugin](https://marketplace.visualstudio.com/items?itemName=bernhard-42.ocp-cad-viewer)
    * [Source](https://github.com/bernhard-42/vscode-ocp-cad-viewer/tree/main)
* [Simple CAM package](https://ocp-freecad-cam.readthedocs.io/en/latest/index.html)

In [280]:
from build123d import *
from ocp_vscode import show, show_object, show_clear
from pathlib import Path
from abc import ABC, abstractmethod

In [281]:
# Load a 3D model from a file
fn_part = 'simple-part.step'
part = import_step(fn_model)
part.label = Path(fn_model).stem

# TODO: Check bounding box dimensions against machine limits
# TODO: Rotate the model if necessary to fit

# Position the model corner at the origin
vec = -part.bounding_box().min
part = part.translate(vec)
part.color = Color("Orange")



In [282]:
# Calculate stock size & placement around model
bbox = part.bounding_box()

# Stock properties
# Stock definitions types:
# Size: fit, fit + margin centered, fit + offsets, fixed + centered, fixed + offsets

# Stock size
# Fit + offsets
# margin = [0,2,0,2,0,2] # [xmin, xmax, ymin, ymax, zmin, zmax]
# corner1 = bbox.min - Vector(margin[0,2,4])
# corner2 = bbox.max + Vector(margin[1,3,5])

# Fit + margin centered
margin = 2
corner1 = bbox.min - Vector(margin, margin, margin)
corner2 = bbox.max + Vector(margin, margin, margin)
vec_to_stock = Vector(margin, margin, margin)

# Stock size
sz = corner2 - corner1

# Create stock box
# TODO: Assign material
# TODO: Assign color
# stock = Box(sz.X,sz.Y,sz.Z)
stock = Solid.make_box(sz.X, sz.Y, sz.Z)
stock.label = "stock"
stock.color = Color("LightGray",alpha=0.5)

# Place part in stock
part = part.translate(vec_to_stock)


In [283]:
# Create a tool
class Tool(BasePartObject):
    def __init__(self, diameter, length):

        self.type = "Flat End Mill"
        self.diameter = diameter
        self.length = length

        solid = Solid.make_cylinder(self.diameter/2, self.length)
        solid.label = self.name

        super().__init__(part = solid)
        self.color = Color("Gray")

        # TODO: Subclass for different tool types.
        # TODO: Add shank/holder

    def __str__(self):
        n = self.name.replace('(','')
        return f"Tool:({self.name}"

    @property
    def name(self)->str:
        return f"{self.type} (d={self._diameter}, L={self.length})"

    @property
    def diameter(self) -> float:
        return self._diameter

    @diameter.setter
    def diameter(self, value: float):
        self._diameter = value

    @property
    def length(self) -> float:
        return self._length

    @length.setter
    def length(self, value:float):
        self._length = value

    @property
    def radius(self)->float:
        return self.diameter / 2


tool = Tool(diameter=3.175, length=25.4)

In [284]:
class Operation(ABC):
    def __init__(self, part=None,
                 tool=None,
                 stock=None,
                 height_safe:float=5.0,
                 speed_feed:int=100,
                 speed_position:int=100,
                 doc:float = 0.3,
                 woc:float = 0.5,
                 ):

        self.part = part
        self.tool = tool
        self.stock = stock
        self.height_safe = height_safe
        self.speed_feed = speed_feed
        self.speed_position = speed_position
        self._toolpath = None
        self._gcode = None

        self.doc = doc
        self.woc = woc

        self._ops = []

    def __str__(self):
        return f"Operation:({self.part}, {self.tool}, {self.stock})"

    @property
    def part(self):
        return self._part

    @part.setter
    def part(self, value):
        self._part = value

    @property
    def tool(self):
        return self._tool

    @tool.setter
    def tool(self, value):
        self._tool = value

    @property
    def stock(self):
        return self._stock

    @stock.setter
    def stock(self, value):
        self._stock = value

    @property
    def height_safe(self):
        return self._height_safe

    @height_safe.setter
    def height_safe(self, value):
        self._height_safe = value

    @property
    def speed_feed(self):
        return self._speed_feed

    @speed_feed.setter
    def speed_feed(self, value):
        self._speed_feed = int(value)

    @property
    def speed_position(self):
        return self._speed_position

    @speed_position.setter
    def speed_position(self, value):
        self._speed_position = int(value)

    @property
    def doc(self)->float:
        """
        Depth of Cut (doc)
        """
        return self._doc

    @doc.setter
    def doc(self,value:float):
        self._doc = value

    @property
    def woc(self)->float:
        return self._woc

    @woc.setter
    def woc(self,value:float):
        self._woc = value

    @property
    def gcode(self) -> str:
        if self._gcode is None:
            self.generate()

        s = '\n'.join(self._ops)

        return s

    @abstractmethod
    def generate(self):
        pass

    def to_wire(self)-> Wire:
        """
        Converts operations to a wire.
        """

        if not self._ops:
            self.generate()

        import re
        expr = "G[01]\s*(X(?P<X>-?\d*\.?\d*))?\s*(Y(?P<Y>-?\d*\.?\d*))?\s*(Z(?P<Z>-?\d*\.?\d*))?"
        expr = re.compile(expr)

        # Initial position
        pos = Vector(0,0,0)

        # Process operations
        edges = []
        for op in self._ops:
            # Parse operation
            # TODO: Assumes G0 or G1 for now
            cmd = "G1"

            res = expr.match(op)

            # Skip ops that don't match
            if res is None:
                continue

            vals = res.groupdict()
            vals = {k:float(v) if v is not None else None for k,v in vals.items()}

            for k,v in vals.items():
                if v is None:
                    if k == 'X':
                        vals[k] = pos.X
                    elif k == 'Y':
                        vals[k] = pos.Y
                    elif k == 'Z':
                        vals[k] = pos.Z

            vec = Vector(**vals)

            # Process command
            if cmd == 'G0':
                # Move to position
                pass
            elif cmd == 'G1':
                # Linear move
                e = Edge.make_line(pos, vec)
                edges.append(e)
                pos = vec

            elif cmd == 'G2':
                # Arc clockwise
                pass
            elif cmd == 'G3':
                # Arc counter-clockwise
                pass

        return Wire(edges)

class OperationFace(Operation):

    # TODO: Face over all of stock or just part
    # TODO: Machining direction: both, climb, standard(?)

    def __init__(self, part=None, tool=None, stock=None, height_safe:float=5.0):
        super().__init__(part, tool, stock, height_safe)

    def generate(self) -> None:
        """
        Generate toolpath for operation.
        """

        # Operations list
        ops = []

        # Precalculated data
        # Save Z position
        safe_z = self.stock.bounding_box().max.Z + self.height_safe
        str_speed_feed = f"F{self.speed_feed:d}"
        str_speed_position = f"F{self.speed_position:d}"
        op_safe_z = f'G0 Z{safe_z:0.3f} {str_speed_position}'

        y_max = stock.bounding_box().max.Y + self.tool.radius*1.1
        y_min = stock.bounding_box().min.Y - self.tool.radius*1.1

        # Move the tool to the first position
        z = self.stock.bounding_box().max.Z
        ops.append(op_safe_z)

        # Move to pre-cut position
        x_start = -self.tool.radius + self.woc
        y_start = y_min
        ops.append(
            f"G0 X{x_start} Y{y_start}"
        )
        z -= self.doc
        ops.append(f"G0 Z{z:0.3f}")
        x = x_start
        y = y_start

        # TODO: Spin up tool

        # Do the facing operations
        # TODO: Loop to top face of part
        x_max = self.stock.bounding_box().max.X
        z_min = self.part.bounding_box().max.Z
        i_pass = 0
        while z >= z_min:
            i_pass += 1
            ops.append(f"; Pass {i_pass}")
            while x < x_max:
                # Move to +Y
                ops.append(f"G1 Y{y_max} {str_speed_feed}")

                # Index over
                x += self.woc
                ops.append(f"G1 X{x} {str_speed_position}")

                # Move to -Y
                ops.append(F"G1 Y{y_min} {str_speed_feed}")

                # Index over
                x += self.woc
                ops.append(f"G1 X{x} {str_speed_position}")

            # Index down
            z -= self.doc
            ops.append(f"G0 Z{z:0.3f} {str_speed_position}")

            # Retrace
            # TODO: need to reverse order of operations.
            x = x_start
            y = y_start
            ops.append(
                f"G0 X{x_start:0.3f} Y{y_start:0.3f} {str_speed_position}"
            )

        # TODO: Add in final pass if needed.
        print('Warning: Final pass not implemented.')

        # Leave the tool at a safe height
        ops.append(op_safe_z)

        self._ops = ops

In [287]:
f = OperationFace(part=part, tool=tool, stock=stock)
f.woc = tool.diameter*0.4
f.doc = 0.75
f.generate()
w = f.to_wire()



In [286]:
# Update display
parts = [w, part, stock, tool]
show(*parts,
     # measure_tools=True,
     #  glass=True,
     )



+++